In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hp\\Documents\\machine_leanrning\\Deep Learning\\MLOPS\\Kidney-Disease-Classification-MLOPS\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd

'c:\\Users\\hp\\Documents\\machine_leanrning\\Deep Learning\\MLOPS\\Kidney-Disease-Classification-MLOPS'

In [ ]:
# update config.yaml

# artifacts_root: artifacts_root

# data_ingestion:
#   root_dir: artifacts/data_ingestion
#   source_URL: http://drive.google.com/uc?export=download&id1j4bfanQ3USJhj9C9txvLooWhWYHWN3Be
#   local_data_file: artifacts/data_ingestion/data.zip
#   unzip_dir: artifacts/data_ingestion



In [5]:
# updata entity (i.e the return type of any func (data ingestion))
# @dataclass means everything after it will be considered as an entity 
# (i.e a kind of class variable from other class that you can access from other files)
# whatever is defined in the config.yaml has to be returned here

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: str
    unzip_dir: str

In [ ]:
# we need to update the constants next
# since they won't be changing, we need to provide a path between params in params & config.yaml

# from pathlib import Path

# CONFIG_FILE_PATH = Path("config/config.yaml")
# PARAMS_FILE_PATH = Path("params.yaml")

In [ ]:
# we update the configuration.py

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
# next, we update the configuration manager
# the function would only return our custom return type

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [12]:
# next, we need to update the components

import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """
        Fetch data from url
        """

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [17]:
# update the pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-05 11:48:04,388]: INFO: common: yaml file: config\config.yaml loaded successfully
[2026-01-05 11:48:04,393]: INFO: common: yaml file: params.yaml loaded successfully
[2026-01-05 11:48:04,393]: INFO: common: created directory at: artifacts
[2026-01-05 11:48:04,401]: INFO: common: created directory at: artifacts/data_ingestion


[2026-01-05 11:48:04,405]: INFO: 2968024951: Downloading data from https://drive.google.com/file/d/1hEIIDJ2ObNRNOlv9J_k2Sdn_9vSUhCs1/view?usp=sharing into file artifacts/data_ingestion/data.zip


Downloading...
From: https://drive.google.com/uc?/export=download&id=1hEIIDJ2ObNRNOlv9J_k2Sdn_9vSUhCs1
To: c:\Users\hp\Documents\machine_leanrning\Deep Learning\MLOPS\Kidney-Disease-Classification-MLOPS\artifacts\data_ingestion\data.zip
100%|██████████| 5.20M/5.20M [00:06<00:00, 846kB/s]

[2026-01-05 11:48:19,706]: INFO: 2968024951: Downloaded data from https://drive.google.com/file/d/1hEIIDJ2ObNRNOlv9J_k2Sdn_9vSUhCs1/view?usp=sharing into file artifacts/data_ingestion/data.zip
